In [ ]:
import re,string
import math, re, string, requests, json
from itertools import product
from inspect import getsourcefile
from os.path import abspath, join, dirname
import nbimporter
from slistener import SListener
import time, tweepy, sys
import pandas as pd
import json
from pandas.io.json import json_normalize
from sklearn.feature_extraction.text import CountVectorizer 

In [ ]:
REMOVE_PUNCTUATION = re.compile('[%s]' % re.escape(string.punctuation))

PUNCTUATION_LIST = [".", "!", "?", ",", ";", ":", "-", "'", "\"",
             "!!", "!!!", "??", "???", "?!?", "!?!", "?!?!", "!?!?"]

NEGATION_WORDS = ["aint", "arent", "cannot", "cant", "couldnt", "darent", "didnt", "doesnt",
 "ain't", "aren't", "can't", "couldn't", "daren't", "didn't", "doesn't",
 "dont", "hadnt", "hasnt", "havent", "isnt", "mightnt", "mustnt", "neither",
 "don't", "hadn't", "hasn't", "haven't", "isn't", "mightn't", "mustn't",
 "neednt", "needn't", "never", "none", "nope", "nor", "not", "nothing", "nowhere",
 "oughtnt", "shant", "shouldnt", "uhuh", "wasnt", "werent",
 "oughtn't", "shan't", "shouldn't", "uh-uh", "wasn't", "weren't",
 "without", "wont", "wouldnt", "won't", "wouldn't", "rarely", "seldom", "despite"]

CONTRASTING_CONJUNCTION = \
{"but", "although", "though", "even though", "even if", "however"}

# Sentiment Incremental score for capitalization approach.
CAPS_INCR = 1.00

# Sentiment score for Punctuation Emphsis symbols
QUESTION_INCREASE = 0.1
EXCLAMATION_INCREASE = 0.25
NEGATION_MODIFIER = -1.00

# Score for Degree Modifiers

SCORE_INCREASE = 0.5
SCORE_DECREASE = -0.5


DEGREE_DICTIONARY = \
    {"100 percent": SCORE_INCREASE, "a good deal": SCORE_INCREASE, "a great deal": SCORE_INCREASE,
     "a lot": SCORE_INCREASE,
     "aboundingly": SCORE_INCREASE, "absolutely": SCORE_INCREASE, "absurdly": SCORE_INCREASE,
     "abundantly": SCORE_INCREASE,
     "admirably": SCORE_INCREASE, "alarmingly": SCORE_INCREASE, "amazingly": SCORE_INCREASE,
     "astronomically": SCORE_INCREASE, "awfully": SCORE_INCREASE, "breathtakingly": SCORE_INCREASE,
     "clearly": SCORE_INCREASE, "completely": SCORE_INCREASE, "considerably": SCORE_INCREASE, "crazy": SCORE_INCREASE,
     "damn": SCORE_INCREASE, "damned": SCORE_INCREASE, "darn": SCORE_INCREASE, "darned": SCORE_INCREASE,
     "decidedly": SCORE_INCREASE, "deeply": SCORE_INCREASE, "deservedly": SCORE_INCREASE, "downright": SCORE_INCREASE,
     "dreadfully": SCORE_INCREASE,  # up to here so far
     "effing": SCORE_INCREASE, "enormously": SCORE_INCREASE,
     "entirely": SCORE_INCREASE, "especially": SCORE_INCREASE, "exceptionally": SCORE_INCREASE,
     "extremely": SCORE_INCREASE,
     "fabulously": SCORE_INCREASE, "flipping": SCORE_INCREASE, "flippin": SCORE_INCREASE,
     "fricking": SCORE_INCREASE, "frickin": SCORE_INCREASE, "frigging": SCORE_INCREASE, "friggin": SCORE_INCREASE,
     "fully": SCORE_INCREASE, "fucking": SCORE_INCREASE,
     "greatly": SCORE_INCREASE, "hella": SCORE_INCREASE, "highly": SCORE_INCREASE, "hugely": SCORE_INCREASE,
     "incredibly": SCORE_INCREASE,
     "intensely": SCORE_INCREASE, "majorly": SCORE_INCREASE, "more": SCORE_INCREASE, "most": SCORE_INCREASE,
     "particularly": SCORE_INCREASE,
     "purely": SCORE_INCREASE, "quite": SCORE_INCREASE, "really": SCORE_INCREASE, "remarkably": SCORE_INCREASE,
     "so": SCORE_INCREASE, "substantially": SCORE_INCREASE,
     "thoroughly": SCORE_INCREASE, "totally": SCORE_INCREASE, "tremendously": SCORE_INCREASE,
     "uber": SCORE_INCREASE, "unbelievably": SCORE_INCREASE, "unusually": SCORE_INCREASE, "utterly": SCORE_INCREASE,
     "very": SCORE_INCREASE,
     "almost": SCORE_DECREASE, "barely": SCORE_DECREASE, "hardly": SCORE_DECREASE, "just enough": SCORE_DECREASE,
     
     "less": SCORE_DECREASE, "little": SCORE_DECREASE, "marginally": SCORE_DECREASE, "occasionally": SCORE_DECREASE,
     "partly": SCORE_DECREASE,
     "scarcely": SCORE_DECREASE, "slightly": SCORE_DECREASE, "somewhat": SCORE_DECREASE,
     "sort of": SCORE_DECREASE, "sorta": SCORE_DECREASE, "sortof": SCORE_DECREASE, "sort-of": SCORE_DECREASE}


In [ ]:
"""
    Check if there is negation contect in the sentence or not.
    Still continuing
"""
def negated(input_words, include_nt=True):
    
    neg_words = []
    neg_words.extend(NEGATION_WORDS)
      
    for word in neg_words:
        if word in input_words:
            return True
    if include_nt:
        for word in input_words:
            if "n't" in word:
                return True
       
    return False

def capitalized(words):
        
        emphasis = False
        capitalized_words = 0

        for word in words:
            if word.isupper():
                capitalized_words += 1
        cap_differential = len(words) - capitalized_words
        if 0 < cap_differential < len(words):
            emphasis = True

        return emphasis


def alter_valence(word, valence, is_cap_diff):
   
    modifier = 0.0
    word_lower = word.lower()
    if word_lower in DEGREE_DICTIONARY:
        modifier = DEGREE_DICTIONARY[word_lower]
        
        if valence < 0:
            modifier = modifier * -1
                    
        if word.isupper() and is_cap_diff:
            if valence > 0:
                modifier += CAPS_INCR
            else: modifier -= CAPS_INCR
    return modifier

In [ ]:
class SentimentAnalyzer(object):
    
    def __init__(self, lexicon_file="New_Lexicon.txt"):
        _this_module_file_path_ = abspath(getsourcefile(lambda:0))
        lexicon_full_filepath = join(dirname(_this_module_file_path_), lexicon_file)
        with open(lexicon_full_filepath) as f:
            self.lexicon_full_filepath = f.read()
        self.lexicon = self.make_lex_dict()
    
    def make_lex_dict(self):
        
        lex_dict = {}
        for line in self.lexicon_full_filepath.split('\n'):
            (word, measure) = line.strip().split('\t')[0:2]
            lex_dict[word] = float(measure)
            
        return lex_dict

    def polarity_scores(self, text):
        
        sentimenttext = SentimentText(text)
        sentiments = []
        words_and_emoticons = sentimenttext.words_and_emoticons
        print(words_and_emoticons)
        
        for item in words_and_emoticons:
            valence = 0
            i = words_and_emoticons.index(item)
            
            if (i < len(words_and_emoticons) - 1 and item.lower() == "kind" and
                        words_and_emoticons[i + 1].lower() == "of") and item.lower() in DEGREE_DICTIONARY:
                sentiments.append(valence)
            
                continue

            sentiments = self.sentiment_score(valence, sentimenttext, item, i, sentiments)
            
        sentiments = self._contains_but(words_and_emoticons, sentiments)
        
        return(sentiments)
    
    def _contains_negation(self, valence, words_and_emoticons, start_i, i):
        
       
        if start_i == 0:
            if negated([words_and_emoticons[i-1]]):
                    valence = valence*NEGATION_MODIFIER                   
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
        if start_i == 1:
            if words_and_emoticons[i-2] == "never" and\
               (words_and_emoticons[i-1] == "so" or
                words_and_emoticons[i-1] == "this"):
                valence = valence*1.5
            elif negated([words_and_emoticons[i-(start_i+1)]]):
                valence = valence*NEGATION_MODIFIER
        if start_i == 2:
            if words_and_emoticons[i-3] == "never" and \
               (words_and_emoticons[i-2] == "so" or words_and_emoticons[i-2] == "this") or \
               (words_and_emoticons[i-1] == "so" or words_and_emoticons[i-1] == "this"):
                valence = valence*1.25
            elif negated([words_and_emoticons[i-(start_i+1)]]):
                valence = valence*NEGATION_MODIFIER

        
        return valence
        
        
    def sentiment_score(self, valence, sentitext, item, i, sentiments):
        
        is_cap_diff = sentitext.is_cap_diff
        words_and_emoticons = sentitext.words_and_emoticons
        item_lowercase = item.lower()
        if item_lowercase in self.lexicon:
            
            valence = self.lexicon[item_lowercase]
            
            if item.isupper() and is_cap_diff:
                if valence > 0:
                    valence += CAPS_INCR
                else:
                    valence -= CAPS_INCR

            for start_i in range(0,4):
                if i > start_i and words_and_emoticons[i-(start_i+1)].lower() not in self.lexicon:
                    s = alter_valence(words_and_emoticons[i-(start_i+1)], valence, is_cap_diff)
                    valence = valence+s
                    valence = self._contains_negation(valence, words_and_emoticons, start_i, i)
                    
        
        sentiments.append(valence)
        return sentiments
            
    
    """
    Algorithm is defined as such:
    The contextual conjective "but" clause works in way to calculate the valence score , algorithm states that the there is 50% 
    reduction in valence score before "but" clause words and 150% increase in the valence score after the "but" clause
    sentence.

    """
    
    def _contains_but(self,words_and_emoticons, sentiments):
       
        CONTRASTING_CONJUNCTION = \
        {"but", "although", "though", "even though", "even if", "however"}
        
        for conj_word in CONTRASTING_CONJUNCTION:
            if conj_word in words_and_emoticons:
                but_index = words_and_emoticons.index(conj_word)
                
                for sentiment in sentiments:
                    sentiment_index = sentiments.index(sentiment)
                    
                    if sentiment_index < but_index:
                        sentiments.pop(sentiment_index)
                        sentiments.insert(sentiment_index, sentiment * 0.5)
                    
                    elif sentiment_index > but_index:
                        sentiments.pop(sentiment_index)
                        sentiments.insert(sentiment_index, sentiment * 1.5)   
                        
        return sentiments          
 

In [ ]:
class SentimentText(object):
    
    def __init__(self, text):
        if not isinstance(text, str):
            text = str(text.encode('utf-8'))
        self.text = text
        self.words_and_emoticons = self._words_and_emoticons()
        self.is_cap_diff = capitalized(self.words_and_emoticons)

    def _words_and_punc(self):
        
        no_punc_text = REMOVE_PUNCTUATION.sub('', self.text)
        words_only = no_punc_text.split()
        
        words_only = set( w for w in words_only if len(w) > 1 )
        punc_before = {''.join(p): p[1] for p in product(PUNCTUATION_LIST, words_only)}
        punc_after = {''.join(p): p[0] for p in product(words_only, PUNCTUATION_LIST)}
        words_punc_dict = punc_before
        words_punc_dict.update(punc_after)
        return words_punc_dict

    def _words_and_emoticons(self):
        
        wes = self.text.split()
        words_punc_dict = self._words_and_punc()
               
        wes = [we for we in wes if len(we) > 1]
        for i, we in enumerate(wes):
            if we in words_punc_dict:
                wes[i] = words_punc_dict[we]
        
        return wes

In [ ]:
# Cleaning of textual data remove URLS's and mentions and hashtags

def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

def strip_all_entities(text):
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)


In [ ]:
# Retrieve text from the json file that is retrived from Tweepy Streaming API

raw_objs_string = open('myprefix.20170601-174814.json').read() 
objs_string = '[%s]'%(raw_objs_string) 
p = re.compile( '}\s*{' )
jsonstr = p.sub( '}\n{', objs_string )
raw_objs_string = jsonstr.replace("}\n{", "},\n{")
objs = json.loads(raw_objs_string) 
json_output = json.dumps(objs)
df = pd.io.json.json_normalize(objs)
textual_tweets = pd.DataFrame(df.text)

In [ ]:
if __name__ == '__main__':
   
    Idioms_sent_score = pd.read_csv("Idioms_Sentiment_Score.csv")
    Idioms_valence = Idioms_sent_score[['idiom','Sentiment_Score']]
    Convert_to_dict = Idioms_valence.set_index('idiom')['Sentiment_Score'].to_dict()  
    analyzer = SentimentAnalyzer()
    
    for t in textual_tweets['text']:
        result = strip_all_entities(strip_links(t))
        result = re.sub('[0-9]+', '', result)
        result = re.sub('RT ','',result)
        result = result.lower()

        vs = analyzer.polarity_scores(result)
        print("{:-<65} {}".format(result, str(vs))) 
        
        #Building Count vectorizer where text are tokenized into n grams.
        
        vectorizer = CountVectorizer(ngram_range=(1,6))
        analyer = vectorizer.build_analyzer()
        result = analyer(result)
        ns = set(result)
        sum_s = float(sum(vs))
        
        #Check if idiom is present in the textual tweets.
        valence_score = 0.0 
        for key, value in Convert_to_dict.items():
            if key in ns:
                valence_score = value
                valence_score = valence_score + sum_s 
                
        ep_count = result.count("!")
        ep_amplifier = 0
        if ep_count > 4:
            ep_count = 4
        ep_amplifier = ep_count * EXCLAMATION_INCREASE    
    
        qm_count = result.count("?")
        qm_amplifier = 0
        if qm_count > 1:
            if qm_count <= 3:
                qm_amplifier = qm_count * QUESTION_INCREASE
            else:
                qm_amplifier = 1.00
            
        punct_emph_amplifier = ep_amplifier+qm_amplifier
        
        # Deriving the sentiment score (positive , negative and neutral sentiment)
        if valence_score > 0:
            valence_score =  valence_score + punct_emph_amplifier
        elif  valence_score < 0:
            valence_score = valence_score -  punct_emph_amplifier
        
        pos_sum = 0.0
        neg_sum = 0.0
        neu_count = 0
    
        for sentiment_score in vs:
            if sentiment_score > 0:
                pos_sum += (float(sentiment_score) +1) 
            if sentiment_score < 0:
                neg_sum += (float(sentiment_score) -1) 
            if sentiment_score == 0:
                neu_count += 1      
        
        if pos_sum > math.fabs(neg_sum):
            pos_sum += (punct_emph_amplifier)
        elif pos_sum < math.fabs(neg_sum):
            neg_sum -= (punct_emph_amplifier)

        total = pos_sum + math.fabs(neg_sum) + neu_count
        pos = math.fabs(pos_sum / total)
        neg = math.fabs(neg_sum / total)
        neu = math.fabs(neu_count / total) 
        sentiment_dict = {"neg" : neg,"neu" : neu,"pos" : pos}
        print(sentiment_dict)